<a href="https://colab.research.google.com/github/CamH123/Thyroid-Disease-Doctor/blob/main/Thyroid_Disease_Modeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Modeling

#### Rice Data Science DEEP

#### Team 3 Thyroid Data

This notebook contains the code we will use to get started modeling the data and making predictions on the data. It will walk through setting up a few multi-class classification models and then give you some space to create your own modeling pipeline.

### Data Cleaning

First we knew to repeat the data cleaning steps we performed in the first workshop below. The primary steps here were fixing some incorrect data, encoding categorical variables using one-hot encoding, and imputing missing data.

In [ ]:
# Importing packages
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from plotnine import *
from sklearn.impute import KNNImputer
from sklearn.model_selection import train_test_split, KFold, cross_validate
from sklearn.metrics import accuracy_score, classification_report

# Code converted to work with polars which is like pandas but faster
import polars as pl
import polars.selectors as cs

# Importing models
from sklearn.linear_model import LogisticRegression # Multinomial Logistic Regression
from sklearn.naive_bayes import GaussianNB # Naive Bayes assuming normally distributed features
from sklearn.ensemble import RandomForestClassifier # Random Forest
from sklearn.ensemble import GradientBoostingClassifier # Gradient Boosting Tree Model

In [ ]:
# Load thyroid data
thyroid_dat = pl.scan_csv("./thyroidDF.csv", infer_schema_length=1000).collect()

# Fix age column
thyroid_dat = thyroid_dat.with_columns(pl.when(pl.col("age") > 100).then(None).otherwise(pl.col("age")).alias("age"))

# Encode boolean variabes as 0's and 1's using to_dummies (more general alternative)
thyroid_dat = thyroid_dat.to_dummies(columns = thyroid_dat[:, thyroid_dat.select(pl.all().n_unique() < 7).row(0)].columns, drop_first = True)

### Data Splitting

Before we do any modeling on the data we need to decide how to split our data. We also need to wait before doing the knn imputation as we want to avoid data leakage in the form of the imputer using information from our test data to affect our training data. A common data splitting scheme when working with typically tabular data is to use an 80-20 train test split and using 5-fold cross validation for our validation set.

In [ ]:
# CODE HERE: Split the data into an 80-20 train test split
target = ["target"]
features = thyroid_dat.drop(target).columns
print(features)
X = thyroid_dat[features]
y = thyroid_dat[target]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)




['age', 'sex_M', 'sex_null', 'on_thyroxine_t', 'query_on_thyroxine_t', 'on_antithyroid_meds_t', 'sick_t', 'pregnant_t', 'thyroid_surgery_t', 'I131_treatment_t', 'query_hypothyroid_f', 'query_hyperthyroid_t', 'lithium_t', 'goitre_t', 'tumor_t', 'hypopituitary_t', 'psych_t', 'TSH_measured_f', 'TSH', 'T3_measured_t', 'T3', 'TT4_measured_t', 'TT4', 'T4U_measured_t', 'T4U', 'FTI_measured_t', 'FTI', 'TBG_measured_t', 'TBG', 'referral_source_STMW', 'referral_source_SVHC', 'referral_source_SVHD', 'referral_source_SVI', 'referral_source_WEST', 'patient_id']


In [ ]:
# CODE HERE: Split the training data into an 95-5 train validation split
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.05, random_state=1)

In [ ]:
# CODE HERE: Create 5 cross validation folds
cv_folds = KFold(n_splits=5, shuffle=True, random_state=1)

#### Finishing data cleaning

In [ ]:
# Initialize KNNImputer with a specified number of neighbors (default is 5)
imputer = KNNImputer(n_neighbors=3)

# Apply the KNN imputer to the DataFrame
temp_cols = thyroid_dat.select(pl.col(pl.Float64, pl.Int64, pl.UInt8)).columns # Save columns as imputer removes them

X_train = pd.DataFrame(imputer.fit_transform(X_train)) # Fit the imputer on the training data but not the test data
X_valid = pd.DataFrame(imputer.transform(X_valid))
X_test = pd.DataFrame(imputer.transform(X_test))

# Add back column names
X_train.columns = temp_cols
X_valid.columns = temp_cols
X_test.columns = temp_cols

### Modeling

We covered binary classification in the previous session where you are trying to classify things as one class or another class. However, our data is split up into 32 different classes, so we will need to keep this in mind with what models we use. I will have you implement five different models of increasing complexity that can perform multiclass classification.

#### Multinomial Logistic Regression

To begin with, we will use logistic regression which has already been covered. However, we will use a variation of traditional logistic regression called multinomial logistic regression that is designed to work with multiple classes. The specifics of how these models differ are largely in their formulas which I will leave to you to research if you are interested.

In [ ]:
# CODE HERE: Implement a Logistic Regression Model, fit it on the training data, and predict the validation and test data
lm_model = LogisticRegression(multi_class='multinomial', solver='lbfgs', max_iter = 500)
lm_model.fit(X_train, y_train)
y_valid_lm = lm_model.predict(X_valid)
y_pred_lm = lm_model.predict(X_test)


/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.


In [ ]:
# Evaluating Logistic Regression Model on Cross-Validation Folds
cross_val_scores = cross_validate(lm_model, X_train, y_train, cv=cv_folds,
                                  scoring=['accuracy', 'precision_weighted', 'recall_weighted', 'f1_weighted'])
avg_cv_scores = pl.DataFrame(cross_val_scores).mean()
avg_cv_scores

/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
/usr/local/lib/python3.10/di

fit_time,score_time,test_accuracy,test_precision_weighted,test_recall_weighted,test_f1_weighted
f64,f64,f64,f64,f64,f64
2.32928,0.123722,0.741033,0.549338,0.741033,0.630888


In [ ]:
# Get classifiction report on validation data
print(classification_report(y_true=y_valid, y_pred=y_valid_lm))

              precision    recall  f1-score   support

           -       0.73      1.00      0.84       267
           A       0.00      0.00      0.00         5
          AK       0.00      0.00      0.00         2
           B       0.00      0.00      0.00         1
         C|I       0.00      0.00      0.00         1
           F       0.00      0.00      0.00        10
           G       0.00      0.00      0.00        12
          GK       0.00      0.00      0.00         4
           I       0.00      0.00      0.00        14
           J       0.00      0.00      0.00         1
           K       0.00      0.00      0.00        19
          KJ       0.00      0.00      0.00         1
           L       0.00      0.00      0.00         4
           M       0.00      0.00      0.00         8
          MK       0.00      0.00      0.00         2
           N       0.00      0.00      0.00         5
           O       0.00      0.00      0.00         2
           R       0.00    

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.


#### Gaussian Naive Bayes

Next we will use a naive bayes model. The gaussian part refers to the assumption for the model that every feature is normally distributed. This obviously is not true for many of our columns, but it is worth evaluating the performance regardless.

In [ ]:
# CODE HERE: Implement a Naive Bayes Model, fit it on the training data, and predict the validation and test data
nb_model = GaussianNB()
nb_model.fit(X_train, y_train)
y_valid_nb = nb_model.predict(X_valid)
y_pred_nb = nb_model.predict(X_test)

/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().


In [ ]:
# Evaluating Naive Bayes Model on Cross-Validation Folds
cross_val_scores = cross_validate(nb_model, X_train, y_train, cv=cv_folds,
                                  scoring=['accuracy', 'precision_weighted', 'recall_weighted', 'f1_weighted'])
avg_cv_scores = pl.DataFrame(cross_val_scores).mean()
avg_cv_scores

/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_cla

fit_time,score_time,test_accuracy,test_precision_weighted,test_recall_weighted,test_f1_weighted
f64,f64,f64,f64,f64,f64
0.025327,0.041186,0.735151,0.550014,0.735151,0.629196


In [ ]:
# Get classifiction report on validation data
print(classification_report(y_true=y_valid, y_pred=y_valid_nb))

              precision    recall  f1-score   support

           -       0.73      1.00      0.84       267
           A       0.00      0.00      0.00         5
          AK       0.00      0.00      0.00         2
           B       0.00      0.00      0.00         1
         C|I       0.00      0.00      0.00         1
           F       0.00      0.00      0.00        10
           G       0.00      0.00      0.00        12
          GK       0.00      0.00      0.00         4
           I       0.00      0.00      0.00        14
           J       0.00      0.00      0.00         1
           K       0.00      0.00      0.00        19
          KJ       0.00      0.00      0.00         1
           L       0.00      0.00      0.00         4
           M       0.00      0.00      0.00         8
          MK       0.00      0.00      0.00         2
           N       0.00      0.00      0.00         5
           O       0.00      0.00      0.00         2
          OI       0.00    

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
/usr/local/lib/python3.10/dist-packages/

#### Random Forest

Next up is a random forest model which is an ensemble model meaning it using many submodels and combines them together to get a better final model.

In [ ]:
# CODE HERE: Implement a Random Forest Model, fit it on the training data, and predict the validation and test data
rf_model = RandomForestClassifier(n_estimators=100, random_state=1)
rf_model.fit(X_train, y_train)
y_valid_rf = rf_model.predict(X_valid)
y_pred_rf = rf_model.predict(X_test)

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


In [ ]:
# Evaluating Random Forest Model on Cross-Validation Folds
cross_val_scores = cross_validate(rf_model, X_train, y_train, cv=cv_folds,
                                  scoring=['accuracy', 'precision_weighted', 'recall_weighted', 'f1_weighted'])
avg_cv_scores = pl.DataFrame(cross_val_scores).mean()
avg_cv_scores

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:1473: DataConversionWa

fit_time,score_time,test_accuracy,test_precision_weighted,test_recall_weighted,test_f1_weighted
f64,f64,f64,f64,f64,f64
1.368674,0.102292,0.939024,0.929168,0.939024,0.931304


In [ ]:
# Get classifiction report on validation data
print(classification_report(y_true=y_valid, y_pred=y_valid_rf))

              precision    recall  f1-score   support

           -       0.96      0.99      0.97       267
           A       0.83      1.00      0.91         5
          AK       0.67      1.00      0.80         2
           B       0.00      0.00      0.00         1
         C|I       0.00      0.00      0.00         1
           F       1.00      1.00      1.00        10
           G       0.86      1.00      0.92        12
          GK       1.00      1.00      1.00         4
           I       0.90      0.64      0.75        14
           J       0.00      0.00      0.00         1
           K       0.85      0.89      0.87        19
          KJ       1.00      1.00      1.00         1
           L       0.60      0.75      0.67         4
           M       1.00      0.75      0.86         8
          MK       0.67      1.00      0.80         2
           N       1.00      0.80      0.89         5
           O       1.00      1.00      1.00         2
           R       0.67    

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.


#### Gradient Boosting Tree

We will also take a look at another ensemble model called a gradient boosting tree model. This model employs an ensembling technique called boosting which uses many different underfitting decision trees to iteratively improve its performance.

In [ ]:
# CODE HERE: Implement a Gradient Boosting Model, fit it on the training data, and predict the validation and test data
gbm_model = GradientBoostingClassifier(n_estimators=100, random_state=1)
gbm_model.fit(X_train, y_train)
y_valid_gbm = gbm_model.predict(X_valid)
y_pred_gbm = gbm_model.predict(X_test)

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().


In [ ]:
# Evaluating Gradient Boosting Model on Cross-Validation Folds
cross_val_scores = cross_validate(gbm_model, X_train, y_train, cv=cv_folds,
                                  scoring=['accuracy', 'precision_weighted', 'recall_weighted', 'f1_weighted'])
avg_cv_scores = pl.DataFrame(cross_val_scores).mean()
avg_cv_scores

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
/usr/local/lib/python3.10/dist-packages/sklearn/metric

fit_time,score_time,test_accuracy,test_precision_weighted,test_recall_weighted,test_f1_weighted
f64,f64,f64,f64,f64,f64
45.354882,0.108329,0.932281,0.932802,0.932281,0.931049


In [ ]:
# Get classifiction report on validation data
print(classification_report(y_true=y_valid, y_pred=y_valid_gbm))

              precision    recall  f1-score   support

           -       0.96      0.97      0.97       267
           A       0.83      1.00      0.91         5
          AK       1.00      0.50      0.67         2
           B       0.00      0.00      0.00         1
         C|I       0.00      0.00      0.00         1
           F       0.90      0.90      0.90        10
           G       0.92      1.00      0.96        12
          GK       0.80      1.00      0.89         4
           I       0.69      0.64      0.67        14
           J       1.00      1.00      1.00         1
           K       0.89      0.89      0.89        19
          KJ       0.50      1.00      0.67         1
           L       0.60      0.75      0.67         4
           M       1.00      0.62      0.77         8
          MK       0.33      0.50      0.40         2
           N       1.00      0.80      0.89         5
           O       0.50      0.50      0.50         2
           R       0.80    

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.


### Improving Our Models

It seems as though the random forest classifier and gradient boosting classifier give similarly decent performanc on the data, however we can certainly improve these models. Here's a few potential strategies:

1. **Perform hyperparameter tuning:** Both models include what are called hyperparameters such as `n_estimators` that can be varied to potentially get better performance. While you can set up some for loops to independently change each hyperparameter over a range of values and evaulate model performance, a better approach is to use the `GridSearchCV` function in sklearn which automates the process. Do however note that hyperparamter tuning is often times a very lengthy process that can take hours.
2. **Split up the target variable**: Looking at the classification reports and based on your intuition it makes sense that any model would struggle to classify some of the rare classes that are combinations of diagnosis such as 'AK' or 'H|K'. One approach you could take to rectify this issue is to create three different columns, the first one for the first diagnosis, the second one for the second diagnosis, and the third for whether it is a pure combination or one with a line through it. After doing this you would simply run three separate classification models for all three target variables.
3. **Take steps to reduce imbalance in the data set**: Nearly 74% of the patients in the data do not have a diagnosis, so our data set has quite a bit of imbalance. One technique you can use to deal with this imbalance is called SMOTE which aims to add synthetic data that samples from the minority classes.
4. **Feature Engineering**: Often times transforming a feature or combing multiple features into one can enhance performance. Think about any combinations of features that may be predictive of diagnosis.

In [ ]:
# CODE HERE: If you have time left over then try improving one of the better performing models


In [ ]:
# Evaluating Your Model on Cross-Validation Folds
cross_val_scores = cross_validate(YOUR_MODEL_HERE, X_train, y_train, cv=cv_folds,
                                  scoring=['accuracy', 'precision_weighted', 'recall_weighted', 'f1_weighted'])
avg_cv_scores = pl.DataFrame(cross_val_scores).mean()
avg_cv_scores